# Introduction

We aggregate the uber pickup data into hourly pick up counts (i.e. 24 samples per day). We look at the hourly pickup counts
as a function of day of the week (i.e. time of the week as we allow "days" like 0.5 which would be Monday noon). Note that we use the convention that the week start with Monday at 0 and Sunday Midnight is 7. We use a periodic model so that our model
transitions smoothly between the end of the week and the start of the week.

Here is a graph of our final regularized model:

![Graph of final model](files/graphs/final_model_counts.svg?santize=true)

This model allows us to pick out intervals where 95% of all pickups lie for a given time of the week.

# Model

We model the data using a regularized negative binomial regression. This is a common model for dealing with data that should be
poisson, but has variances that are too large. The idea is that the conditional distribution of `y = hourly pickup count`
given a value `x = (continuous) day of the week` is a latent poisson mixture where the poisson mean of y is distributed by
a latent gamma distribution. The resulting conditional distribution gives `y | x` a negative binomial distribution.

Let us give a brief description of the model. Technical details may be found in
[Greene *Functional Forms for the Negative Binomial Model for Count Data*](http://people.stern.nyu.edu/wgreene/EconomicsLetters-NBP.pdf).
The idea is that there is a latent variable `g` that is distributed as a gamma distribution with mean 1 and variance to be
determined. Given a value of `x` and `g`, `y` is distrbuted as a poisson distribution with mean `m(x) * g`, where `m(x)` is log-linear, i.e. `log m(x) = Ax + B`.

There is more than one way to choose a variance for `g`. We will look at two methods:
* `nb1` : The variance of `g` is chosen such that `Variance(y|x) = m(x) + alpha * m(x)` for some constant parameter `alpha`.
* `nb2` : The variance of `g` is chosen such that `Varaince(y|x) = m(x) + alpha * m(x)**2` for some constant parameter `alpha`.

## Regularization

We will look at using regularization weights that are:
* constant for different fourier inputs.
* are weighted according to the derivatives of the fourier inputs.

Weighting the regularization penalties in a way related to the derivatives of the fourier terms is related to punishing the
amount of "wiggle" introduced by the term.


# Model Selection

We need to choose a type of regularization weight and whether to use `nb1` or `nb2` negative binomial model.

We use cross-validation where we score our hold-out set on its mean log-likelihood for the trained model. Here are the results:

![All Model Scores](files/graphs/loglikelihood_all_models.svg?sanitize=true)

We see that the unregularized model has significantly worse log-likelihood scores. Let us take a closer look at the regularized
models:

![Regularized Model Scores](files/graphs/loglikelihood_regularized_models.svg?sanitize=true)

Based on the scores and the fact that a rough measure of correlation between graphing the log-mean vs log-variance for
binned statistics, we choose to use the derivative-based regularization with `nb1` negative binomial model.

# Final Model

We train the final model on the entire dataset. The graph of the final result is above. We also get the
following for the Fourier Coefficients sizes:

![Model Coefficient Sizes](files/graphs/fourier_coeff_sizes.svg?sanitize=true)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

from my_src import my_graphs, my_periodic

# When saving svg plots, we use regular text for plot text. This stops the default
# behavior of outputting curves for each letter of text and saves a lot of memory
# per graph.

plt.rcParams['svg.fonttype'] = 'none'
sns.set() # Let seaborn handle graphs.
violin_plot_grid_size = 20
violin_plot_fig_size = (12, 10)

In [ ]:
data = pd.read_csv('data/uber-raw-data-aug14.csv')

In [ ]:
print(data.info())
data.head()

In [ ]:
data['Date/Time'] = pd.to_datetime(data['Date/Time'])
print(data.info())
data.head()

In [ ]:
reduced_date = data['Date/Time'].apply(lambda x : (x.month, x.day, x.hour))
reduced_date = pd.DataFrame(reduced_date.tolist(), columns = ['month', 'day', 'hour'])
reduced_date.head()

In [ ]:
# Now get hourly counts by day and hour.

hour_counts = (reduced_date.groupby(['day', 'hour'])
                .count()
                .rename(columns =  {'month' : 'count'}))
hour_counts.head()

In [ ]:
initial_day = 4
day_of_week = hour_counts.index.to_series().apply(lambda x : (x[0] - 1 + initial_day) % 7)
print('Before adding in hourly part\n', day_of_week.value_counts().sort_index())
day_of_week += hour_counts.index.get_level_values('hour') / 24
day_of_week.head()

In [ ]:
plt.scatter(day_of_week, hour_counts['count'])
plt.xlabel('Day of the Week (to the Hour)')
plt.ylabel('Hourly Pickup Count')
plt.title('Hourly Pickup Counts Through the Week')
plt.tight_layout()
plt.savefig('graphs/continuous_hourly_counts.svg')
plt.show()

Let's take a look at the hourly pickup counts for the different times of the week.

![Hourly Pickup Counts vs Continuous Day of the Week](files/graphs/continuous_hourly_counts.svg?sanitize=true)

# Rough Statistics Using Bins

In [ ]:
# Get bin statistics.

bin_size_hours = 2
bin_size = 1 / 24 * bin_size_hours
bins = np.arange(0, 7 + 1/24, bin_size)
bin_cut = pd.cut(day_of_week, bins = bins)
bin_mean = hour_counts.groupby(bin_cut).mean().sort_index()
bin_variance = (hour_counts.groupby(bin_cut).std()**2).sort_index()

In [ ]:
# Plot the bin variance vs the bin mean. Use logarithmic scales.

bin_model = LinearRegression()#fit_intercept = False)
bin_model.fit(np.log10(bin_mean), np.log10(bin_variance).values.reshape(-1))
print('log_bin_variance = ', '{:.3f}'.format(bin_model.coef_[0]), ' * log_bin_mean + ',
      '{:.3f}'.format(bin_model.intercept_))

line_x = np.linspace(np.log10(bin_mean.min()), np.log10(bin_mean.max()), 4)
plt.scatter(np.log10(bin_mean), np.log10(bin_variance))
plt.plot(line_x, bin_model.predict(line_x), color = 'red')
plt.xlabel('Log10(Bin Mean)')
plt.ylabel('Log10(Bin Variance)')
plt.title('Comparison of Bin Variances to Bin Means')
plt.tight_layout()
plt.savefig('graphs/bin_var_bin_mean.svg')
plt.show()

Comparing the logarithms of the bin variances to the logarithms of the bin means, we see that
```
log_bin_variance = 1.251 * log_bin_mean + 0.352
```

![Log10(Bin_Variance) vs Log10(Bin_Mean)](files/graphs/bin_var_bin_mean.svg?sanitize=true)

# Make Periodic Features

In [ ]:
# Make the periods for weekly and daily trends.

n_weekly_fourier = 30
n_daily_fourier = 11 # For daily fourier, don't go over 11; the data is sampled by the hour.
                     # Remember that dimension doubles and can't exceed 24. Also we are
                     # already adding constant functions in addition to fourier stuff.

weekly_frequency = np.arange(1, n_weekly_fourier + 1) # per week
daily_frequency = 7 * np.arange(1, n_daily_fourier + 1) # per week
all_frequency = np.concatenate([weekly_frequency, daily_frequency])
all_frequency = np.unique(all_frequency)

all_periods = 7.0 / all_frequency
print(all_periods.shape)
all_frequency

In [ ]:
# Make the periodic features and the alphas for regularization.

y = hour_counts['count']
X, alpha_base = my_periodic.make_periodic(day_of_week, all_periods, return_alphas = 'L1')

column_names = ['freq_' + str(freq) for freq in all_frequency]
column_names = [[col + '_c', col + '_s'] for col in column_names]
column_names = np.array(column_names).reshape(-1)
X = pd.DataFrame(X, index = day_of_week.index, columns = column_names)
print(X.shape)
X.head()

# Do Non-Regularized NegativeBinomial

In [ ]:
import statsmodels
from statsmodels.discrete.discrete_model import NegativeBinomial
import scipy.stats as sts

In [ ]:
best_scores = {'unregularized' : []}

In [ ]:
def translate_nbinom_params(statsmodels_params, means, loglike_method):
    # statsmodels negative binomial fit results seems to have no implementation for
    # finding the log-likelihood of samples in X and y. So we will use the scipy.stats
    # implementation of the negative binomial distribution to find the log-likelihood.
    # Translation between two libraries:
    #      nb2:
    #      scipy.stats.nbinom n = statsmodels alpha
    #      scipy.stats.nbinom p = statsmodels theta / (theta + mean(X))
    #                           = 1 / (1 + alpha * mean(X))
    
    theta = 1 / statsmodels_params['alpha']
    if loglike_method == 'nb2':
        ns = theta
        ps = 1 / (1 + statsmodels_params['alpha'] * means)
    elif loglike_method == 'nb1':
        ns = theta * means
        ps = theta / (1 + theta)
    else:
        raise Exception('loglike_method not a valid value.')
    return {'n' : ns, 'p' : ps}

In [ ]:
# Make function for computing log-likelihood of negative binomial model.

def find_log10_likelihood(fitted_model, X, y, loglike_method, add_constant = True):
    # statsmodels negative binomial fit results seems to have no implementation for
    # finding the log-likelihood of samples in X and y. So we will use the scipy.stats
    # implementation of the negative binomial distribution to find the log-likelihood.
    # Translation between two libraries:
    #      scipy.stats.nbinom n = statsmodels alpha
    #      scipy.stats.nbinom p = statsmodels theta / (theta + mean(X))
    #                           = 1 / (1 + alpha * mean(X))
    if add_constant:
        X = statsmodels.tools.add_constant(X)
    means = fitted_model.predict(X)
    nbinom_kwargs = translate_nbinom_params(fitted_model.params, means, loglike_method)   
        
    log_lls = sts.nbinom.logpmf(k = y, **nbinom_kwargs) / np.log(10)
    return log_lls

In [ ]:
def get_nbinomial_start_params(X, y, loglike_method, add_constant = True):
    _, n_zeros = X.shape
    if add_constant:
        n_zeros = n_zeros + 1
        
    y_mean = y.mean()
    if loglike_method == 'nb2':
        initial_alpha = (y.std()**2 - y_mean) / y_mean**2
    elif loglike_method == 'nb1':
        initial_alpha = (y.std()**2 - y_mean) / y_mean
    elif loglike_method != 'geometric':
        raise Exception('loglike_method value not recognized.')
        
    if loglike_method != 'geometric':
        start_params = np.zeros(n_zeros + 1)
        start_params[-1] = initial_alpha
    else:
        start_params = np.zeros(n_zeros)
        
    start_params[0] = np.log(y_mean)
    return start_params

In [ ]:
def get_nbinom_intervals(fitted_model, X, percentage, loglike_method, add_constant = True):
    # statsmodels negative binomial fit results seems to have no implementation for
    # finding the log-likelihood of samples in X and y. So we will use the scipy.stats
    # implementation of the negative binomial distribution to find the log-likelihood.
    # Translation between two libraries:
    #      nb2:
    #      scipy.stats.nbinom n = statsmodels alpha
    #      scipy.stats.nbinom p = statsmodels theta / (theta + mean(X))
    #                           = 1 / (1 + alpha * mean(X))
    if add_constant:
        X = statsmodels.tools.add_constant(X)
    means = fitted_model.predict(X)
    nbinom_kwargs = translate_nbinom_params(fitted_model.params, means, loglike_method)
    
    intervals = sts.nbinom.interval(percentage, **nbinom_kwargs)
    return intervals


In [ ]:
splitter = KFold(n_splits = 5, shuffle = True)
start_params = get_nbinomial_start_params(X, y, loglike_method = 'nb1')

best_scores['unregularized'] = {key : [] for key in ['train', 'test']}

for split_ind in splitter.split(X):
    keys = ['train', 'test']
    X_split = {key : statsmodels.tools.add_constant(X.loc[ind, :], has_constant = 'add') for key, ind in zip(keys, split_ind)}
    y_split = {key : y.loc[ind, :] for key, ind in zip(keys, split_ind)}
    negative_binomial = NegativeBinomial(endog = y_split['train'], exog = X_split['train'], loglike_method = 'nb1')
    fitted_model = negative_binomial.fit(start_params = start_params)
    
    for key in best_scores['unregularized']:
            log_ll = find_log10_likelihood(fitted_model, X_split[key], y_split[key],
                                           loglike_method = 'nb1').mean()
            best_scores['unregularized'][key].append(log_ll)
            
for key in best_scores['unregularized']:
    best_scores['unregularized'][key] = np.array(best_scores['unregularized'][key])
    

In [ ]:
# Look for maximum score.

for position, scores in enumerate(best_scores['unregularized'].values()):
    plt.boxplot(scores, positions = [position])

plt.xticks([0, 1], best_scores['unregularized'].keys())
plt.ylabel('Mean Log10 Likelihood')
plt.title('Log10 Likelihoods for UnRegularized Fit')
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize = (15, 5))
plt.scatter(day_of_week, hour_counts)
labels = []
percentile = 0.95

p_days = np.linspace(0, 7, 7 * 24 * 3)
plot_X = my_periodic.make_periodic(p_days, all_periods, return_alphas = None)
plot_X = statsmodels.tools.add_constant(plot_X, has_constant = 'add')

intervals = get_nbinom_intervals(fitted_model, plot_X, percentile, loglike_method = 'nb1', add_constant = False)
means = fitted_model.predict(plot_X)
plt.plot(p_days, means, color = 'red')
for endpoint_i in range(2):
    plt.plot(p_days, intervals[endpoint_i], color = 'purple')
        
plt.legend(['model mean', 'model ' + str(int(percentile *100)) + '% interval'])
plt.xlabel('Day of Week')
plt.ylabel('Hourly Pick Up Count')
plt.title('Unregularized Model Hourly Pickup Counts')
plt.tight_layout()
plt.savefig('graphs/unregularized_model_counts.svg')
plt.show()

Let's take a look at the graph of one of the unregularized models.

![Unregularized Model](files/graphs/unregularized_model_counts.svg?sanitize=true)

# Make Function for Cross Validation

In [ ]:
def do_gridsearch_nbinom(X, y, alphas, loglike_method, **fit_kwargs):
    negative_binomial = NegativeBinomial(endog = y['train'],
                                         exog = X['train'],
                                         loglike_method = loglike_method)
    search_scores = {key : [] for key in X}
    start_params = get_nbinomial_start_params(X['train'], y['train'], loglike_method, add_constant = False)
    
    for alpha in alphas: 
        fitted_model = negative_binomial.fit_regularized(alpha = alpha, 
                                                         start_params = start_params,
                                                         **fit_kwargs)
        for key in search_scores:
            log_ll = find_log10_likelihood(fitted_model, X[key], y[key],
                                           loglike_method = loglike_method).mean()
            search_scores[key].append(log_ll)
        # Reset start_params to last fitted values for speed-up.
        start_params = np.array(fitted_model.params)
        
    return search_scores
    
def do_cv_nbinom(X, y, log_alpha_sizes, base_alphas, loglike_method, splitter, add_constant = True, **fit_kwargs):
    if add_constant:
        base_alphas = np.concatenate([[0], # No penalty for bias term.
                                      base_alphas, # Penalties for coefficients.
                                      [0]]) # No penalty for alpha parameter related to latent
                                            # Poisson distribution.
        X = statsmodels.tools.add_constant(X, has_constant = 'add')
    else:
        base_alphas = np.concatenate([base_alphas, # Penalties for coefficients.
                                     [0]]) # No penalty for alpha parameter related to latent
                                           # Poisson distribution.
    cv_scores = {'train' : [], 'test' : []}   
    
    for split_ind in splitter.split(X, y):
        split_ind = {'train' : X.index[split_ind[0]],
                     'test' : X.index[split_ind[1]]}
        X_split = {key : X.loc[ind] for key, ind in split_ind.items()} 
        y_split = {key : y.loc[ind] for key, ind in split_ind.items()}
        alphas = (base_alphas * np.exp(log_size) for log_size in log_alpha_sizes)
        search_scores = do_gridsearch_nbinom(X_split, y_split, alphas, loglike_method, **fit_kwargs)
        for key in cv_scores:
            cv_scores[key].append(search_scores[key])
        
    for key in cv_scores:
        cv_scores[key] = np.array(cv_scores[key])
    return cv_scores
        

# Do Regularized Fit With Flat Penalty Scaling

In [ ]:
best_log_alpha = {'flat' : None}

In [ ]:
splitter = KFold(n_splits = 5, shuffle = True)
log_alpha_sizes = np.linspace(6, 8, 15)
flat_alpha_base = np.full(alpha_base.shape, 1.0)
flat_alpha_base = flat_alpha_base / np.linalg.norm(flat_alpha_base)
cv_scores = do_cv_nbinom(X, y, log_alpha_sizes = log_alpha_sizes, base_alphas = flat_alpha_base, 
                         loglike_method = 'nb1', 
                         splitter = splitter,
                         trim_mode = 'off',
                         maxiter = 300,
                         disp = False)

In [ ]:
# Look for maximum score.
plt.figure(figsize = (15, 5))
plt.subplot(1, 2, 1)
plt.title('All Scores')
for key, color in zip(cv_scores.keys(), ['blue', 'red']):
    plt.plot(log_alpha_sizes, cv_scores[key].T, color = color)

plt.subplot(1, 2, 2)
plt.title('Mean Scores')
for key, color in zip(cv_scores.keys(), ['blue', 'red']):
    plt.plot(log_alpha_sizes, cv_scores[key].mean(axis = 0), color = color)
    
plt.legend(cv_scores.keys())
plt.show()

plt.plot(log_alpha_sizes, cv_scores['test'].mean(axis = 0))
plt.show()

In [ ]:
best_cutoff = 7.25
best_ind = (log_alpha_sizes < best_cutoff).sum() - 1
best_log_alpha['flat'] = log_alpha_sizes[best_ind]
best_scores['flat'] = {key : cv_scores[key][:, best_ind] for key in cv_scores}
best_log_alpha['flat']

In [ ]:
# Look for maximum score.

for position, scores in enumerate(best_scores['flat'].values()):
    plt.boxplot(scores, positions = [position])

plt.xticks([0, 1], best_scores['flat'].keys())
plt.ylabel('Mean Log10 Likelihood')
plt.title('Log10 Likelihoods for Best Flat Fit')
plt.tight_layout()
plt.show()

# Do Regularized Fit With Regularization Depending on Derivatives

In [ ]:
# Make function to estimate risk between model distribution and real distribution.
# Note we don't calculute the square term for the true distribution as it is
# independent of the model.

def find_risk(fitted_model, X, y, loglike_method, add_constant = True, n_draws_per_point = 50):
    if add_constant:
        X = statsmodels.tools.add_constant(X)
    means = fitted_model.predict(X)
    nbinom_kwargs = translate_nbinom_params(fitted_model.params, means, loglike_method)
    
    # Approximate the cross-term by turning integral in over true distribution into a sample
    # mean of model probability.
    
    cross_term = sts.nbinom.pmf(k = y, **nbinom_kwargs).mean()
    
    # Approximate the square-term by making draws from the model distribution for each X_i.
    kwargs_broadcast = {key : np.array(param).reshape(-1, 1) for key, param in nbinom_kwargs.items()}
    y_draws = sts.nbinom.rvs(**kwargs_broadcast, size = y.shape + (n_draws_per_point,))
    model_ps = sts.nbinom.pmf(k = y_draws, **kwargs_broadcast)
    square_term = model_ps.mean()
    
    return square_term - 2 * cross_term

## NBD1

Here we look at when the `variance = mean + alpha * mean`.

In [ ]:
splitter = KFold(n_splits = 5, shuffle = True)
log_alpha_sizes = np.linspace(1.5, 7, 15)
log_alpha_sizes = np.linspace(6, 10, 15)
cv_scores = do_cv_nbinom(X, y, log_alpha_sizes = log_alpha_sizes, base_alphas = alpha_base, 
                         loglike_method = 'nb1', 
                         splitter = splitter,
                         disp = False,
                         trim_mode = 'off',
                         maxiter = 200)#,
                         #qc_tol = 1.0)

In [ ]:
# Look for maximum score.
plt.figure(figsize = (15, 5))
plt.subplot(1, 2, 1)
plt.title('All Scores')
for key, color in zip(cv_scores.keys(), ['blue', 'red']):
    plt.plot(log_alpha_sizes, cv_scores[key].T, color = color)

plt.subplot(1, 2, 2)
plt.title('Mean Scores')
for key, color in zip(cv_scores.keys(), ['blue', 'red']):
    plt.plot(log_alpha_sizes, cv_scores[key].mean(axis = 0), color = color)
    
plt.legend(cv_scores.keys())
plt.show()

plt.plot(log_alpha_sizes, cv_scores['test'].mean(axis = 0))
plt.show()

In [ ]:
best_cutoff = 8.5
best_ind = (log_alpha_sizes < best_cutoff).sum() - 1
best_log_alpha['derivative_nbd1'] = log_alpha_sizes[best_ind]
best_scores['derivative_nbd1'] = {key : cv_scores[key][:, best_ind] for key in cv_scores}
best_log_alpha['derivative_nbd1']

In [ ]:
# Look for maximum score.

for position, scores in enumerate(best_scores['derivative_nbd1'].values()):
    plt.boxplot(scores, positions = [position])

plt.xticks([0, 1], best_scores['derivative_nbd1'].keys())
plt.ylabel('Mean Log10 Likelihood')
plt.title('Log10 Likelihoods for Best Derivative Fit')
plt.tight_layout()
plt.show()

## NBD2

Try model with `variance = mean + alpha * mean**2`.

In [ ]:
splitter = KFold(n_splits = 5, shuffle = True)
log_alpha_sizes = np.linspace(1.5, 7, 15)
log_alpha_sizes = np.linspace(6, 10, 15)
cv_scores = do_cv_nbinom(X, y, log_alpha_sizes = log_alpha_sizes, base_alphas = alpha_base, 
                         loglike_method = 'nb2', 
                         splitter = splitter,
                         disp = False,
                         trim_mode = 'off',
                         maxiter = 200)#,
                         #qc_tol = 1.0)

In [ ]:
# Look for maximum score.
plt.figure(figsize = (15, 5))
plt.subplot(1, 2, 1)
plt.title('All Scores')
for key, color in zip(cv_scores.keys(), ['blue', 'red']):
    plt.plot(log_alpha_sizes, cv_scores[key].T, color = color)

plt.subplot(1, 2, 2)
plt.title('Mean Scores')
for key, color in zip(cv_scores.keys(), ['blue', 'red']):
    plt.plot(log_alpha_sizes, cv_scores[key].mean(axis = 0), color = color)
    
plt.legend(cv_scores.keys())
plt.show()

plt.plot(log_alpha_sizes, cv_scores['test'].mean(axis = 0))
plt.show()

In [ ]:
best_cutoff = 8.5
best_ind = (log_alpha_sizes < best_cutoff).sum() - 1
best_log_alpha['derivative_nbd2'] = log_alpha_sizes[best_ind]
best_scores['derivative_nbd2'] = {key : cv_scores[key][:, best_ind] for key in cv_scores}
best_log_alpha['derivative_nbd2']

In [ ]:
# Look for maximum score.

for position, scores in enumerate(best_scores['derivative_nbd2'].values()):
    plt.boxplot(scores, positions = [position])

plt.xticks([0, 1], best_scores['derivative_nbd2'].keys())
plt.ylabel('Mean Log10 Likelihood')
plt.title('Log10 Likelihoods for Best Derivative Fit')
plt.tight_layout()
plt.show()

# Graph Best Results For All Models

In [ ]:
fig = plt.figure(figsize = (10, 5))
position = 1
tick_labels = []
for model in best_scores:
    for key, scores in best_scores[model].items():
        plt.boxplot(scores, positions = [position])
        tick_labels.append(model + '_' + key)
        position += 1
plt.xticks(np.arange(1, len(tick_labels) + 1), tick_labels, rotation = 25)
plt.ylabel('Log10 Likelihood')
plt.title('Log10 Likelihood For Different Methods')
plt.tight_layout()
plt.savefig('graphs/loglikelihood_all_models.svg')
plt.show()

fig = plt.figure(figsize = (10, 5))
position = 1
tick_labels = []
for model in list(best_scores.keys())[1:]:
    for key, scores in best_scores[model].items():
        plt.boxplot(scores, positions = [position])
        tick_labels.append(model + '_' + key)
        position += 1
plt.xticks(np.arange(1, len(tick_labels) + 1), tick_labels, rotation = 25)
plt.ylabel('Log10 Likelihood')
plt.title('Log10 Likelihood For Different Regularized Methods')
plt.tight_layout()
plt.savefig('graphs/loglikelihood_regularized_models.svg')
plt.show()

Let's take a look at the results for the different choices of models. First let's look at all of the models:

![Log10 Likelihood of All Models](files/graphs/loglikelihood_all_models.svg?sanitize=true)

We see that the regularized models perform much better than the non-regularized model.

![Log10 Likelihood of Regularized Models](files/graphs/loglikelihood_regularized_models.svg?sanitize=true)

There isn't considerably much difference in performance for the different regularized models. However, the `nbd1` model
seems like it should be a better fit than the `nbd2` model based on the correlations between the binned means and binned variances. Also, it seems intuitive that using regularization parameters depending on the derivatives makes more sense than
just flat regularization parameters. So we will use the regularized weights related to the derivatives of the function.

In [ ]:
# Make the final model and do a fit to all of the data.

best_alpha_size = np.exp(best_log_alpha['derivative_nbd1'])
final_negative_binomial = NegativeBinomial(endog = y,
                                           exog = statsmodels.tools.add_constant(X, has_constant = 'add'),
                                           loglike_method = 'nb1')
alpha_const_base = np.concatenate([[0], alpha_base, [0]])
start_params = get_nbinomial_start_params(X, y, loglike_method = 'nb1')
final_result = final_negative_binomial.fit_regularized(alpha = best_alpha_size * alpha_const_base,
                                                       max_iter = 300,
                                                       start_params = start_params,
                                                       trim_mode = 'off')

In [ ]:
# Graph the coefficients of the final model.

for key in ['const', 'alpha']:
    print('final_result.params[\'' + key + '\'] = ', 
          '{:.2f}'.format(final_result.params[key]))

fig = plt.figure(figsize = (15, 5))
plot_frequency = np.stack([all_frequency, all_frequency], axis = -1).reshape(-1)
plt.plot(all_frequency, np.abs(final_result.params[1:-1:2]))
plt.plot(all_frequency, np.abs(final_result.params[2:-1:2]))
plt.scatter(all_frequency, np.abs(final_result.params[1:-1:2]))
plt.scatter(all_frequency, np.abs(final_result.params[2:-1:2]))
plt.xlabel('Frequency (per week)')
plt.legend(['Cosine', 'Sine'])
plt.ylabel('Size')
plt.title('Sizes of Fourier Coefficients for Final Model')
plt.tight_layout()
plt.savefig('graphs/fourier_coeff_sizes.svg')
plt.show()

Let's take a look at the sizes of the coefficients of the final model.

![Fourier Coefficient Sizes](files/graphs/fourier_coeff_sizes.svg?sanitize=true)

In [ ]:
fig = plt.figure(figsize = (15, 5))
plt.scatter(day_of_week, hour_counts)
labels = []
percentile = 0.95

p_days = np.linspace(0, 7, 7 * 24 * 3)
plot_X = my_periodic.make_periodic(p_days, all_periods, return_alphas = None)
plot_X = statsmodels.tools.add_constant(plot_X, has_constant = 'add')

intervals = get_nbinom_intervals(final_result, plot_X, percentile, loglike_method = 'nb1', add_constant = False)
means = final_result.predict(plot_X)
plt.plot(p_days, means, color = 'red')
for endpoint_i in range(2):
    plt.plot(p_days, intervals[endpoint_i], color = 'purple')
        
plt.legend(['model mean', 'model ' + str(int(percentile *100)) + '% interval'])
plt.xlabel('Day of Week')
plt.ylabel('Hourly Pick Up Count')
plt.title('Final Model Hourly Pickup Counts')
plt.tight_layout()
plt.savefig('graphs/final_model_counts.svg')
plt.show()

Let's take a look at how the final model matches the data and get the interval for where 95% of all counts lie (depending
on the day of the week).

![Final Model Hourly Pickup Counts](files/graphs/final_model_counts.svg?sanitize=true)